In [1]:
%load_ext autoreload
%autoreload 2

https://webstruct.readthedocs.io/en/latest/tutorial.html

In [ ]:
!pip install webstruct >/dev/null
!pip install lxml >/dev/null
!pip install scikit-learn >/dev/null


In [ ]:
import webstruct
trees = webstruct.load_trees("train/*.html", webstruct.WebAnnotatorLoader())

In [ ]:
list(trees)

In [ ]:
html_tokenizer = webstruct.HtmlTokenizer()
X, y = html_tokenizer.tokenize(trees)

def token_identity(html_token):
    return {'token': html_token.token}

def token_isupper(html_token):
    return {'isupper': html_token.token.isupper()}

def parent_tag(html_token):
    return {'parent_tag': html_token.parent.tag}

def border_at_left(html_token):
    return {'border_at_left': html_token.index == 0}

from webstruct.feature_extraction import HtmlFeatureExtractor
feature_extractor = HtmlFeatureExtractor(
    token_features = [
        token_identity,
        token_isupper,
        parent_tag,
        border_at_left
    ]
)

In [ ]:
features = feature_extractor.fit_transform(X)

## named entity extraction

https://www.kdnuggets.com/2018/10/named-entity-recognition-classification-scikit-learn.html/2

In [ ]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles

In [1]:
!pip install scikit-learn==0.23.1
!pip install pandas
!pip install sklearn_crfsuite >/dev/null

In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('./data/ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)
sentences = getter.sentences

In [5]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [6]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [18]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/home/grigory/anaconda3/envs/crf/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [24]:
y_train

[['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['B-gpe'],
 ['B-per'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-per'],
 ['B-org'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['B-per'],
 ['B-geo'],
 ['O'],
 ['B-org'],
 ['O'],
 ['O'],
 ['B-org'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['B-per'],
 ['B-per'],
 ['B-geo'],
 ['O'],
 ['B-org'],
 ['B-gpe'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-tim'],
 ['O'],
 ['O'],
 ['B-org'],
 ['O'],
 ['B-geo'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['O'],
 ['B-tim'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['B-geo'],
 ['O'],
 ['O'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],


In [19]:
X_train[10]

[{'bias': 1.0,
  'word.lower()': 'the',
  'word[-3:]': 'The',
  'word[-2:]': 'he',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'DT',
  'postag[:2]': 'DT',
  'BOS': True,
  'EOS': True}]

In [25]:
y_train

[['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['B-gpe'],
 ['B-per'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-per'],
 ['B-org'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['B-per'],
 ['B-geo'],
 ['O'],
 ['B-org'],
 ['O'],
 ['O'],
 ['B-org'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['B-per'],
 ['B-per'],
 ['B-geo'],
 ['O'],
 ['B-org'],
 ['B-gpe'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-tim'],
 ['O'],
 ['O'],
 ['B-org'],
 ['O'],
 ['B-geo'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-gpe'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['O'],
 ['B-tim'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-geo'],
 ['B-geo'],
 ['O'],
 ['O'],
 ['O'],
 ['B-per'],
 ['O'],
 ['O'],


In [26]:
X_train = [
    [dict(a=1,b=4)],
    [dict(a=2,b=3)],
    [dict(a=1,b=3)],
    [dict(a=1,b=2)]
]
y_train = [
    ['a'], ['b'], ['a'], ['a']
]
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/home/grigory/anaconda3/envs/crf/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [30]:
X_val = [
    [dict(a=1,b=4)],
    [dict(a=2,b=3)],
    [dict(a=-1,b=3)],
    [dict(a=500,b=3)],
    [dict(a=1,b=20)],
]
y_val = [
    ['a'], ['b'], ['a'], ['a']
]

crf.predict(X_val)

[['a'], ['b'], ['a'], ['b'], ['a']]

In [23]:
crf.transition_features_

{}

In [20]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:

Top unlikely transitions:


https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-use-conll-2002-data-to-build-a-ner-system

### Py depta(Update не получается поставить из за scrapely и gcc error которая не гуглитс)

In [ ]:
# !sudo apt-get install python-dev

In [ ]:
# !sudo apt-get install -y libxml2-dev libxslt1-dev

In [1]:
# !pip3 install scrapely

In [2]:
# !pip install -i https://pypi.anaconda.org/scrapinghub/label/dev/simple pydepta
# !pip install git+https://github.com/scrapinghub/pydepta.git

In [3]:
import pydepta

ModuleNotFoundError: No module named 'pydepta'

In [33]:
from pydepta import Depta
d = Depta()
url1 = 'http://www.iens.nl/restaurant/12229/nijmegen-pasta-e-fagioli'
seed = d.extract(url=url1)[8]
seed.as_plain_texts()[0]

ModuleNotFoundError: No module named 'pydepta'

### Scrapely ставится на python 3.7.4 с некотрой ошибкой

https://github.com/scrapy/scrapely/issues/112

In [2]:
import scrapely

In [9]:
from scrapely import Scraper
s = Scraper()


In [4]:
url1 = 'http://pypi.python.org/pypi/w3lib/1.1'
data = {'name': 'w3lib 1.1', 'author': 'Scrapy project', 'description': 'Library of web-related functions'}
s.train(url1, data)

In [5]:
url2 = 'http://pypi.python.org/pypi/Django/1.3'
s.scrape(url2)
[{u'author': [u'Django Software Foundation &lt;foundation at djangoproject com&gt;'],
  u'description': [u'A high-level Python Web framework that encourages rapid development and clean, pragmatic design.'],
  u'name': [u'Django 1.3']}]

[{'description': ['A high-level Python Web framework that encourages rapid development and clean, pragmatic design.'],
  'name': ['\n        Django 1.3\n      ']}]

In [23]:
s.train_from_htmlpage??

In [17]:
s = Scraper()

url3 = 'https://pypi.org/project/Scrapy/'
train_dict = {'abstract': ['Python 3.6+', 'Works on Linux, Windows, macOS, BSD']}
s.train(url3, train_dict)

In [26]:
from scrapely import best_match

best_match('kek')

<function scrapely.template.best_match.<locals>.func(fragment, page)>

In [19]:
url4 = 'https://pypi.org/project/selenium/'
s.scrape(url4)

[{'abstract': ['Python 2.7, 3.4+']}]